# Problema


 Uma empresa fara entregas de encomendas utilizando um caminhao e um drone. O caminhao
 parte de um deposito e ao chegar em um cliente, a entrega da encomenda e feita e o caminhao
 pode se tornar uma base para disparos do drone que fara entregas a outros clientes que estejam
 em seu raio de opera cao, entregando uma encomenda por voo e sempre voltando ao caminhao
 apos o t ermino das entregas. O tempo de percurso entre cada localidade a ser visitada quando
 feita de caminhao e K vezes maior que se feita com o drone. O dados do problema apresentam
 as coordenadas cartesianas das localidades, o raio de acao do drone e o fator de velocidade
 K. Utilize a distância euclidiana entre as localidades tambem como o tempo de trajeto feito
 de drone. Determine modelo de otimiza cao que minimize o tempo total para a finalizacao das
 entregas e retorno ao deposito. Esse é um problema clássico de roteamento com veículos heterogêneos (caminhão + drone), uma variação do problema conhecido como Flying Sidekick Traveling Salesman Problem (FSTSP). O objetivo é minimizar o tempo total de uma rota em que:

O caminhão parte do depósito, realiza entregas diretas e serve de base para o drone.

O drone só pode entregar uma encomenda por voo e deve retornar ao caminhão após cada entrega.

O drone tem um raio máximo de operação e é mais rápido (fator K vezes) que o caminhão.

Vamos resolver esse problema usando o Gurobi e um modelo de programação inteira mista (MIP) simplificado.

📌 Explicação Geral do Modelo
Definições:

𝑁
N: conjunto de locais (incluindo depósito).

𝐷
⊂
𝑁
D⊂N: clientes que podem ser atendidos por drone.

𝑅
R: raio de operação do drone.

𝐾
K: fator de velocidade do caminhão em relação ao drone.

𝑑
𝑖
𝑗
d
ij
​
 : distância euclidiana entre os pontos
𝑖
i e
𝑗
j.

Variáveis:

𝑥
𝑖
𝑗
∈
{
0
,
1
}
x
ij
​
 ∈{0,1}: 1 se o caminhão vai diretamente de
𝑖
i para
𝑗
j.

𝑦
𝑖
∈
{
0
,
1
}
y
i
​
 ∈{0,1}: 1 se o cliente
𝑖
i for atendido pelo drone.

𝑡
t: tempo total de operação (variável contínua a ser minimizada).

Função Objetivo:
Minimizar
𝑡
t: tempo total de rota do caminhão e entregas com drone.

Restrições:

Cada cliente é atendido uma única vez, por caminhão ou drone.

O drone só pode ser lançado em clientes dentro do raio
𝑅
R.

O drone entrega e retorna ao caminhão antes da próxima parada do caminhão.

O percurso do caminhão é um circuito iniciado e terminado no depósito.

# Solver


In [ ]:
!pip install gurobipy

In [ ]:
import gurobipy as gp
import random
import math
import folium
import matplotlib.pyplot as plt
import os
import time


def distancia(i, j):
  return math.sqrt((j[0] - i[0])**2 + (j[1] - i[1])**2)

def solve_tsp_drone(R, K, coordenadas, file_name):
        # R = 0
        start_time = time.time()
        coordenadas.append(coordenadas[0])

        n = len(coordenadas)  # número de cidades (nós)

        options = {
            "WLSACCESSID": "f92c5a8b-e73c-4a49-9b04-8cd13ef70cb3",
            "WLSSECRET": "79dde2d8-9e43-4737-b91e-1a8527c10224",
            "LICENSEID": 2660671,
        }

        with gp.Env(params=options) as env, gp.Model(env=env) as modelo:

          #Variáveis para rotas do caminhão
          x = modelo.addVars(n, n,  vtype = gp.GRB.BINARY)

          #Variáveis para rotas do drone
          y = modelo.addVars(n, n, vtype = gp.GRB.BINARY)

          #Variáveis para eliminação de subrotas
          u = modelo.addVars(n)

          # Função Objetivo
          modelo.setObjective((sum(K*(distancia(coordenadas[i],coordenadas[j]) * x[i,j] )
                                  for i in range(n) for j in range(n) if i!=j)) +
                              (sum(2*(distancia(coordenadas[i],coordenadas[j]) * y[i,j])
                                  for i in range(n) for j in range(n) if i!=j)))

          # Primeira restrição
          # Definição dos nós visitados pelo caminhão
          # Ou ele visita, com uma entrada e uma saída (1 - 1 = 0)
          # Ou ele não visita, sem entrada nem saída (0 - 0 = 0)
          r1 = modelo.addConstrs(
              (sum((x[i, k]) for i in range(0, n-1) if i!=k)) -
              (sum((x[k, j]) for j in range(1, n) if k!=j)) == 0
              for k in range(1, n-1)
          )

          # Segunda restrição
          # Garante que o caminhão saia do depósito
          r2 = modelo.addConstr(
              (sum(x[0, j] for j in range(1, n-1)) == 1)
          )

          # Terceira restrição
          # Garante que o nó foi visitado pelo caminhão OU pelo drone
          r3 = modelo.addConstrs(
              (sum(x[i, j] for i in range(0, n-1) if i != j)) + (sum(y[i, j] for i in range(1, n-1) if i != j)) == 1
              for j in range(1, n-1)
          )

          # Quarta restrição
          # Garante que o drone saia somente de onde o caminhão visitou
          r4 = modelo.addConstrs(
              (sum(y[k, j] for j in range(1, n) if k != j)) <=
              n*(sum(x[i, k] for i in range(0, n-1) if i != k))
              for k in range(1, n-1)
          )

          # Quinta restrição
          # Garante que o drone só visite pontos que estejam dentro do seu raio
          r5 = modelo.addConstrs(
              (y[i, j] == 0
              for i in range(n)
              for j in range(n)
              if i != j and distancia(coordenadas[i], coordenadas[j]) > R)
          )


          # Sexta restrição
          # Eliminação de sub-rotas
          # Exige que a rota feche somente depois de passar por todos os nós
          r6 = modelo.addConstrs(
              u[j] >= u[i] + n * x[i,j] - n + 1
              for i in range(n-1) for j in range(1, n) if i != j
          )


          # Limpando o terminal
          modelo.setParam("OutputFlag", 1)

          # Resolvendo o modelo
          modelo.optimize()

          # Tempo de execução
          end_time = time.time()
          execution_time = end_time - start_time
          print("Tempo de execução: {:.2f} segundos\n".format(execution_time))


          # Valor ótimo
          print("Valor ótimo = {:.3f}\n".format(modelo.objVal))
          print("Distância aproximada em quilômetros = {:.3f}km\n".format(modelo.objVal * 60 * 1.852))
          # Custo por modal
          custo_caminhao = sum(
              K * distancia(coordenadas[i], coordenadas[j]) * x[i, j].x
              for i in range(n) for j in range(n) if i != j
          )

          custo_drone = sum(
              2 * distancia(coordenadas[i], coordenadas[j]) * y[i, j].x
              for i in range(n) for j in range(n) if i != j
          )

          custo_total = custo_caminhao + custo_drone

          percentual_caminhao = (custo_caminhao / custo_total) * 100 if custo_total > 0 else 0
          percentual_drone = (custo_drone / custo_total) * 100 if custo_total > 0 else 0
          # Nós atendidos por drone
            # desconsidera depósito e retorno
          nos_por_drone = sum(
              1 for j in range(1, n-1)
              if any(y[i, j].x > 0.5 for i in range(n) if i != j)
          )
          percentual_nos_drone = (nos_por_drone / (n - 1)) * 100

          # Gap da solução (se aplicável)
          gap = modelo.MIPGap if modelo.MIPGap is not None else 0.0

          # Status
          status_map = {
              gp.GRB.OPTIMAL: "Ótimo",
              gp.GRB.INFEASIBLE: "Inviável",
              gp.GRB.UNBOUNDED: "Ilimitado",
              gp.GRB.TIME_LIMIT: "Tempo limite",
          }
          status_nome = status_map.get(modelo.Status, f"Outro ({modelo.Status})")

          # Resultados
          print(f"Custo caminhão: {custo_caminhao:.2f}")
          print(f"Custo drone: {custo_drone:.2f}")
          print(f"% nós atendidos por drone: {percentual_nos_drone:.1f}%")
          print(f"Gap: {gap * 100:.2f}%")
          print(f"Status da solução: {status_nome}")
          print(f"% custo caminhão: {percentual_caminhao:.1f}%")
          print(f"% custo drone: {percentual_drone:.1f}%")



          plot_rota(file_name, n, x, y, coordenadas)

# Impressão Rotas


In [ ]:
import matplotlib.pyplot as plt

def plot_rota(file_name, n, x, y, coordenadas):

    # 1) Identifica arcos percorridos
    arcos_caminhao = [[i, j] for i in range(n) for j in range(n) if i != j and round(x[i, j].x) == 1]
    arcos_drone = [[i, j] for i in range(n) for j in range(n) if i != j and round(y[i, j].x) == 1]

    print("Arcos Caminhão:", arcos_caminhao)
    print("Arcos Drone:", arcos_drone)

    # 2) Constrói sequência de visita do caminhão
    rota_caminhao = []
    visitados = set()
    atual = 0
    while True:
        rota_caminhao.append(atual)
        visitados.add(atual)
        prox = [j for i, j in arcos_caminhao if i == atual and j not in visitados]
        if not prox:
            break
        atual = prox[0]

    primeiro_ponto = rota_caminhao[0]
    ultimo_ponto = rota_caminhao[-1]

    # 3) Índices únicos de nós visitados
    indices_truck = {idx for arc in arcos_caminhao for idx in arc}
    indices_drone = {idx for arc in arcos_drone for idx in arc}

    # 4) Prepara coordenadas
    xs_truck = [coordenadas[i][0] for i in indices_truck]
    ys_truck = [coordenadas[i][1] for i in indices_truck]

    xs_drone = [coordenadas[i][0] for i in indices_drone]
    ys_drone = [coordenadas[i][1] for i in indices_drone]

    # 5) Plota
    plt.figure(figsize=(12, 8))
    plt.scatter(xs_truck, ys_truck, marker='o', label='Caminhão')
    plt.scatter(xs_drone, ys_drone, marker='x', label='Drone')

    # Arcos do caminhão
    for i, j in arcos_caminhao:
        x0, y0 = coordenadas[i]
        x1, y1 = coordenadas[j]
        plt.plot([x0, x1], [y0, y1], '-o', color='yellow', linewidth=1)

    # Arcos dos drones
    for i, j in arcos_drone:
        x0, y0 = coordenadas[i]
        x1, y1 = coordenadas[j]
        plt.plot([x0, x1], [y0, y1], '--x', color='red', linewidth=1)

    # Destaques especiais
    x_ini, y_ini = coordenadas[primeiro_ponto]
    plt.plot(x_ini, y_ini, marker='s', color='green', markersize=10, label='Início (Depósito)')

    x_fim, y_fim = coordenadas[ultimo_ponto]
    plt.plot(x_fim, y_fim, marker='P', color='purple', markersize=10, label='Último Ponto')

    plt.xlabel('Coord X')
    plt.ylabel('Coord Y')
    plt.legend()
    plt.title('Nós visitados por Caminhão (o) e Drone (x)')

    # 6) Salva e mostra
    plt.savefig(f"/content/images/{file_name}.png", dpi=300, bbox_inches="tight")
    plt.show()


# Leitura de arquivos


In [ ]:
import os
import glob

def parse_input(path):
    coordenadas = []
    with open(path, 'r') as f:
        R = int(f.readline().strip())
        K = int(f.readline().strip())
        for line in f:
            line = line.strip()
            if not line:
                continue
            parts = line.split()
            if len(parts) != 2:
                raise ValueError(f"Linha inválida em {path}: {line!r}")
            x, y = map(int, parts)
            coordenadas.append([x, y])
    return R, K, coordenadas

def parse_all_inputs(dir_path):
    results = {}
    pattern = os.path.join(dir_path, '*.txt')
    for filepath in glob.glob(pattern):
        nome = os.path.splitext(os.path.basename(filepath))[0]
        R, K, coords = parse_input(filepath)
        results[nome] = {
            'R': R,
            'K': K,
            'coordenadas': coords
        }
    return results

# Variações

## Vários drones

In [ ]:
def solve_tsp_many_drones(R, K, coordenadas, file_name, num_drones=3):
    import gurobipy as gp
    import math
    import os
    import matplotlib.pyplot as plt

    def distancia(i, j):
        return math.sqrt((j[0] - i[0])**2 + (j[1] - i[1])**2)

    coordenadas.append(coordenadas[0])
    n = len(coordenadas)

    options = {
        "WLSACCESSID": "f92c5a8b-e73c-4a49-9b04-8cd13ef70cb3",
        "WLSSECRET": "79dde2d8-9e43-4737-b91e-1a8527c10224",
        "LICENSEID": 2660671,
    }

    with gp.Env(params=options) as env, gp.Model(env=env) as modelo:

        # Caminhão
        x = modelo.addVars(n, n, vtype=gp.GRB.BINARY)

        # Vários drones
        y = modelo.addVars(n, n, num_drones, vtype=gp.GRB.BINARY)

        # Subtour elimination
        u = modelo.addVars(n)

        # Objetivo
        modelo.setObjective(
            gp.quicksum(K * distancia(coordenadas[i], coordenadas[j]) * x[i, j]
                        for i in range(n) for j in range(n) if i != j)
            +
            gp.quicksum(2 * distancia(coordenadas[i], coordenadas[j]) * y[i, j, k]
                        for k in range(num_drones)
                        for i in range(n)
                        for j in range(n)
                        if i != j),
            gp.GRB.MINIMIZE
        )

        # R1: Caminhão entra e sai se visita
        modelo.addConstrs(
            (gp.quicksum(x[i, k] for i in range(n-1) if i != k)
             - gp.quicksum(x[k, j] for j in range(1, n) if k != j)) == 0
            for k in range(1, n-1)
        )

        # R2: Caminhão sai do depósito
        modelo.addConstr(gp.quicksum(x[0, j] for j in range(1, n-1)) == 1)

        # R3: Cada ponto deve ser visitado por caminhão OU algum drone
        modelo.addConstrs(
            (gp.quicksum(x[i, j] for i in range(n-1) if i != j)
             + gp.quicksum(y[i, j, k] for k in range(num_drones) for i in range(1, n-1) if i != j)) == 1
            for j in range(1, n-1)
        )

        # R4: Drone só parte de ponto visitado pelo caminhão
        modelo.addConstrs(
            num_drones * gp.quicksum(x[i, k] for i in range(n-1) if i != k) >=
            gp.quicksum(y[k, j, d] for j in range(1, n) if j != k for d in range(num_drones))
            for k in range(1, n-1)
        )

        # R5: Drone só pode ir até o raio R
        modelo.addConstrs(
            y[i, j, k] == 0
            for i in range(n)
            for j in range(n)
            for k in range(num_drones)
            if i != j and distancia(coordenadas[i], coordenadas[j]) > R
        )

        # R6: Subtour elimination (somente para caminhão)
        modelo.addConstrs(
            u[j] >= u[i] + n * x[i, j] - n + 1
            for i in range(n-1)
            for j in range(1, n)
            if i != j
        )

        modelo.setParam("OutputFlag", 0)
        modelo.optimize()

        print("Valor ótimo = {:.3f}".format(modelo.objVal))
        print("Distância aprox. em km = {:.3f}km".format(modelo.objVal * 60 * 1.852))

        plot_rote_many_drones(file_name, n, x, y, coordenadas, num_drones)


In [ ]:
import matplotlib.pyplot as plt

def plot_rote_many_drones(file_name, n, x, y, coordenadas, num_drones):
    """
    Plota o mapa das rotas do caminhão e dos drones com base nas variáveis de decisão x e y.

    Parâmetros:
    - file_name: nome do arquivo para salvar a imagem
    - n: número de nós
    - x: variáveis binárias do caminhão (dict de tuplas)
    - y: variáveis binárias dos drones (dict de triplas)
    - coordenadas: lista de tuplas (x, y) com as coordenadas dos nós
    - num_drones: número de drones
    """

    # 1) Arcos percorridos
    arcos_caminhao = [[i, j] for i in range(n) for j in range(n)
                      if i != j and round(x[i, j].x) == 1]

    arcos_drone = [[[i, j] for i in range(n) for j in range(n)
                    if i != j and round(y[i, j, k].x) == 1]
                   for k in range(num_drones)]

    print("Arcos Caminhão:", arcos_caminhao)
    for k, arcos in enumerate(arcos_drone):
        print(f"Arcos Drone {k}:", arcos)

    # 2) Índices únicos de nós visitados
    indices_truck = {idx for arc in arcos_caminhao for idx in arc}
    indices_drone = {idx for k in range(num_drones)
                     for arc in arcos_drone[k] for idx in arc}

    # 3) Coordenadas
    xs_truck = [coordenadas[i][0] for i in indices_truck]
    ys_truck = [coordenadas[i][1] for i in indices_truck]

    xs_drone = [coordenadas[i][0] for i in indices_drone]
    ys_drone = [coordenadas[i][1] for i in indices_drone]

    # 4) Plot
    plt.figure(figsize=(8, 6))
    plt.scatter(xs_truck, ys_truck, marker='o', color='blue', label='Caminhão')
    plt.scatter(xs_drone, ys_drone, marker='x', color='red', label='Drones')

    # Caminhão
    for i, j in arcos_caminhao:
        x0, y0 = coordenadas[i]
        x1, y1 = coordenadas[j]
        plt.plot([x0, x1], [y0, y1], '-o', color='blue', linewidth=1)

    # Drones
    cores = ['red', 'green', 'orange', 'purple', 'brown', 'gray']
    for k in range(num_drones):
        cor = cores[k % len(cores)]
        for i, j in arcos_drone[k]:
            x0, y0 = coordenadas[i]
            x1, y1 = coordenadas[j]
            plt.plot([x0, x1], [y0, y1], '--x', color=cor, linewidth=1, label=f'Drone {k}' if i == arcos_drone[k][0][0] else None)

    plt.xlabel('Coord X')
    plt.ylabel('Coord Y')
    plt.legend()
    plt.title(f'Rotas do Caminhão e {num_drones} Drones')

    # 5) Salva e mostra
    os.makedirs("/content/images", exist_ok=True)
    plt.savefig(f"/content/images/{file_name}.png", dpi=300, bbox_inches="tight")
    plt.show()


## Drone pode encontrar o Caminhão em outro ponto

# Main

In [ ]:
if __name__ == "__main__":
    folder = '/content/instancia'
    process_files = parse_all_inputs(folder)

    for name, data in process_files.items():
        R = data['R']
        K = data['K']
        coords = data['coordenadas']
        print(f"\nProcessando {name}.txt — R={R}, K={K}, {len(coords)} pontos")

        solve_tsp_drone(R, K, coords, name)